In [5]:
# simple_campaign_monitor_with_export.py
# Minimal campaign daily monitor + CSV + Excel export + email alerts
# Email subject = "Campaign report alerts - <date>" where <date> = (latest_date_in_data - 1 day)
# Requirements: pandas, openpyxl (for Excel export)
# Optional for email: set SMTP env vars (ALERT_SMTP_*)

import os
from datetime import timedelta
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText

# -------------------- CONFIG (edit these) --------------------
INPUT_PATH ="D:\Downloads\query_result_2025-10-16T05_46_10.207636705Z.xlsx" # Excel or CSV
HIST_DAYS = 30                # how many days to use for "historical" average
MIN_IMPRESSIONS = 100         # ignore tiny rows
MIN_CLICKS = 5
# thresholds: fraction relative to historical average. Positive=alert on increase, Negative=alert on drop.
THRESHOLDS = {
    "ctr": 0.50,              # CTR >= hist*(1+0.5)
    "cpc": 0.50,              # CPC >= hist*(1+0.5)
    "conversion_rate": -0.30, # CVR <= hist*(1-0.3) (drop)
    "cpa": 0.50               # CPA increase
}
# Column names in your file. Change here if your file uses different names.
DATE_COL = "date"
CAMPAIGN_ID = "campaign_id"
CAMPAIGN_NAME = "campaign_name"
ACCOUNT_COL = "account_id"   # optional; will be created as "unknown_account" if missing
BRAND_COL = "brand"          # optional; will be created as "unknown_brand" if missing
OUTPUT_DIR = "simple_monitor_output"
# ------------------------------------------------------------

EPS = 1e-9

def safe_div(a, b):
    return np.where(b == 0, 0.0, a / b)

def load_data(path):
    if str(path).lower().endswith((".xls", ".xlsx")):
        return pd.read_excel(path, engine="openpyxl")
    return pd.read_csv(path)

def ensure_columns(df):
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    # find date column if needed
    if DATE_COL not in df.columns:
        candidates = [c for c in df.columns if "date" in c.lower()]
        if candidates:
            df.rename(columns={candidates[0]: DATE_COL}, inplace=True)
    df[DATE_COL] = pd.to_datetime(df[DATE_COL]).dt.normalize()

    # numeric columns
    for c in ["impressions", "clicks", "cost", "conversions", "conversion_value"]:
        if c not in df.columns:
            df[c] = 0
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

    # campaign id/name mapping or generate
    if CAMPAIGN_ID not in df.columns:
        alt = [c for c in df.columns if "campaign" in c.lower() and "id" in c.lower()]
        if alt:
            df.rename(columns={alt[0]: CAMPAIGN_ID}, inplace=True)
        else:
            df[CAMPAIGN_ID] = df.index.astype(str)
    if CAMPAIGN_NAME not in df.columns:
        alt = [c for c in df.columns if "campaign" in c.lower() and "name" in c.lower()]
        if alt:
            df.rename(columns={alt[0]: CAMPAIGN_NAME}, inplace=True)
        else:
            df[CAMPAIGN_NAME] = df[CAMPAIGN_ID].astype(str)

    # optional account/brand
    if ACCOUNT_COL not in df.columns:
        df[ACCOUNT_COL] = "unknown_account"
    if BRAND_COL not in df.columns:
        df[BRAND_COL] = "unknown_brand"

    return df

def compute_daily(df):
    group = [DATE_COL, ACCOUNT_COL, BRAND_COL, CAMPAIGN_ID, CAMPAIGN_NAME]
    agg = df.groupby(group, as_index=False).agg({
        "impressions":"sum", "clicks":"sum", "cost":"sum",
        "conversions":"sum", "conversion_value":"sum"
    })
    agg["ctr"] = safe_div(agg["clicks"], agg["impressions"])
    agg["cpc"] = safe_div(agg["cost"], agg["clicks"])
    agg["conversion_rate"] = safe_div(agg["conversions"], agg["clicks"])
    agg["cpa"] = safe_div(agg["cost"], agg["conversions"])
    agg["roi"] = safe_div(agg["conversion_value"], agg["cost"])
    return agg

def historical_average(metrics_df, target_date, days):
    hist_end = pd.to_datetime(target_date) - pd.Timedelta(days=1)
    hist_start = hist_end - pd.Timedelta(days=days-1)
    hist = metrics_df[(metrics_df[DATE_COL] >= hist_start) & (metrics_df[DATE_COL] <= hist_end)]
    if hist.empty:
        return pd.DataFrame()  # nothing
    cols = [ACCOUNT_COL, BRAND_COL, CAMPAIGN_ID]
    hist_avg = hist.groupby(cols).agg({
        "impressions":"mean","clicks":"mean","cost":"mean",
        "conversions":"mean","conversion_value":"mean",
        "ctr":"mean","cpc":"mean","conversion_rate":"mean","cpa":"mean","roi":"mean"
    }).reset_index()
    # rename historical cols with _hist suffix for clarity
    hist_avg = hist_avg.rename(columns={c: f"{c}_hist" for c in hist_avg.columns if c not in cols})
    return hist_avg

def detect_breaches(target_day_df, hist_avg_df):
    merged = target_day_df.merge(hist_avg_df, on=[ACCOUNT_COL, BRAND_COL, CAMPAIGN_ID], how="left")
    # percent changes relative to historical avg
    merged["ctr_pct"] = (merged["ctr"] - merged.get("ctr_hist", 0)) / (merged.get("ctr_hist", 0) + EPS)
    merged["cpc_pct"] = (merged["cpc"] - merged.get("cpc_hist", 0)) / (merged.get("cpc_hist", 0) + EPS)
    merged["conv_rate_pct"] = (merged["conversion_rate"] - merged.get("conversion_rate_hist", 0)) / (merged.get("conversion_rate_hist", 0) + EPS)
    merged["cpa_pct"] = (merged["cpa"] - merged.get("cpa_hist", 0)) / (merged.get("cpa_hist", 0) + EPS)

    alerts = []
    for _, r in merged.iterrows():
        if r["impressions"] < MIN_IMPRESSIONS or r["clicks"] < MIN_CLICKS:
            continue
        breach_items = []
        if "ctr" in THRESHOLDS and pd.notnull(r.get("ctr_hist")):
            if r["ctr_pct"] >= THRESHOLDS["ctr"]:
                breach_items.append(("ctr", r["ctr"], r.get("ctr_hist"), r["ctr_pct"]))
        if "cpc" in THRESHOLDS and pd.notnull(r.get("cpc_hist")):
            if r["cpc_pct"] >= THRESHOLDS["cpc"]:
                breach_items.append(("cpc", r["cpc"], r.get("cpc_hist"), r["cpc_pct"]))
        if "conversion_rate" in THRESHOLDS and pd.notnull(r.get("conversion_rate_hist")):
            th = THRESHOLDS["conversion_rate"]
            if th < 0 and r["conv_rate_pct"] <= th:
                breach_items.append(("conversion_rate_drop", r["conversion_rate"], r.get("conversion_rate_hist"), r["conv_rate_pct"]))
            elif th >= 0 and r["conv_rate_pct"] >= th:
                breach_items.append(("conversion_rate_rise", r["conversion_rate"], r.get("conversion_rate_hist"), r["conv_rate_pct"]))
        if "cpa" in THRESHOLDS and pd.notnull(r.get("cpa_hist")):
            if r["cpa_pct"] >= THRESHOLDS["cpa"]:
                breach_items.append(("cpa", r["cpa"], r.get("cpa_hist"), r["cpa_pct"]))
        if breach_items:
            alerts.append({
                "account": r[ACCOUNT_COL],
                "brand": r[BRAND_COL],
                "campaign_id": r[CAMPAIGN_ID],
                "campaign_name": r[CAMPAIGN_NAME],
                "date": r[DATE_COL].date().isoformat(),
                "impressions": int(r["impressions"]),
                "clicks": int(r["clicks"]),
                "cost": float(r["cost"]),
                "conversions": int(r["conversions"]),
                "conversion_value": float(r["conversion_value"]),
                "breach_details": "; ".join([f"{m}: {cur:.4g} vs {hist:.4g} ({pct:+.1%})" for m,cur,hist,pct in breach_items])
            })
    return pd.DataFrame(alerts), merged

def send_email(subject, html_body):
    host = os.getenv("ALERT_SMTP_HOST")
    port = int(os.getenv("ALERT_SMTP_PORT", "587"))
    user = os.getenv("ALERT_SMTP_USER")
    pwd = os.getenv("ALERT_SMTP_PASS")
    from_addr = os.getenv("ALERT_FROM")
    to_addrs = os.getenv("ALERT_TO")
    if not all([host, user, pwd, from_addr, to_addrs]):
        print("SMTP not configured via env vars. Skipping email.")
        return False
    to_list = [t.strip() for t in to_addrs.split(",") if t.strip()]
    msg = MIMEText(html_body, "html")
    msg["Subject"] = subject
    msg["From"] = from_addr
    msg["To"] = ", ".join(to_list)
    try:
        s = smtplib.SMTP(host, port, timeout=20)
        s.starttls()
        s.login(user, pwd)
        s.sendmail(from_addr, to_list, msg.as_string())
        s.quit()
        print("Email sent to:", to_list)
        return True
    except Exception as e:
        print("Failed to send email:", e)
        return False

def export_reports(daily_all, daily_target, alerts_df, target_date):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    date_str = target_date.strftime("%Y%m%d")
    # CSVs
    daily_all.to_csv(os.path.join(OUTPUT_DIR, f"daily_all_{date_str}.csv"), index=False)
    daily_target.to_csv(os.path.join(OUTPUT_DIR, f"daily_target_{date_str}.csv"), index=False)
    alerts_df.to_csv(os.path.join(OUTPUT_DIR, f"alerts_{date_str}.csv"), index=False)
    # Single Excel workbook with sheets
    try:
        excel_path = os.path.join(OUTPUT_DIR, f"campaign_report_{date_str}.xlsx")
        with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
            daily_all.to_excel(writer, sheet_name="daily_all", index=False)
            daily_target.to_excel(writer, sheet_name="daily_target", index=False)
            alerts_df.to_excel(writer, sheet_name="alerts", index=False)
        print("Saved CSVs and Excel:", os.path.abspath(OUTPUT_DIR))
    except Exception as e:
        print("Excel export skipped (openpyxl missing or error):", e)

def run_monitor():
    df_raw = load_data(INPUT_PATH)
    df = ensure_columns(df_raw)
    daily = compute_daily(df)

    if daily.empty:
        print("No data found. Exiting.")
        return

    latest_date = daily[DATE_COL].max()
    target_date = latest_date  # using latest date as the date we check
    previous_day_for_subject = (latest_date - pd.Timedelta(days=1)).date().isoformat()

    # prepare target day data
    target_day = daily[daily[DATE_COL] == target_date].copy()
    if target_day.empty:
        print("No rows for latest date:", latest_date.date(), "Exiting.")
        return

    hist_avg = historical_average(daily, target_date, HIST_DAYS)
    alerts_df, merged_full = detect_breaches(target_day, hist_avg)

    # export CSV + Excel
    export_reports(daily, target_day, alerts_df, target_date)

    # compose email
    subject = f"Campaign report alerts - {previous_day_for_subject}"
    if alerts_df.empty:
        body_html = f"<p>Hi Team,</p><p>No campaign breaches detected for <b>{target_date.date()}</b>.</p>"
    else:
        body_html = f"<p>Hi Team,</p><p>Campaign breaches detected for <b>{target_date.date()}</b> (historical window: last {HIST_DAYS} days).</p>"
        body_html += alerts_df.to_html(index=False, escape=False)
        body_html += "<p>CSV and Excel reports saved in folder: {}</p>".format(os.path.abspath(OUTPUT_DIR))

    send_email(subject, body_html)
    print("Done. Export folder:", os.path.abspath(OUTPUT_DIR))

if __name__ == "__main__":
    run_monitor()


C:\Users\ilmar\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Saved CSVs and Excel: C:\Users\ilmar\simple_monitor_output
SMTP not configured via env vars. Skipping email.
Done. Export folder: C:\Users\ilmar\simple_monitor_output


In [6]:
!pip install pandas google-ads openpyxl


   ---------------------------------------- 0.0/17.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/17.8 MB 5.3 MB/s eta 0:00:04
   ---------------------------------------- 0.2/17.8 MB 5.3 MB/s eta 0:00:04
   - -------------------------------------- 0.5/17.8 MB 2.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.3/17.8 MB 5.3 MB/s eta 0:00:04
   --- ------------------------------------ 1.7/17.8 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.2/17.8 MB 6.5 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/17.8 MB 5.9 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/17.8 MB 5.9 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/17.8 MB 5.9 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/17.8 MB 5.9 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/17.8 MB 5.9 MB/s eta 0:00:03
   ----- ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 6.33.0 which is incompatible.


In [ ]:
# campaign_monitor_google_ads.py
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# Try to import google-ads client; fail gracefully with instructions
try:
    from google.ads.googleads.client import GoogleAdsClient
except Exception as e:
    GoogleAdsClient = None

import smtplib
from email.mime.text import MIMEText

# ---------------- CONFIG ----------------
# Google Ads: list of customer IDs (strings, e.g. "1234567890")
CUSTOMER_IDS = ["INSERT_CUSTOMER_ID_HERE"]

# Date window to fetch (we'll fetch last N days up to yesterday)
LOOKBACK_DAYS = 90

# Monitor settings
HIST_DAYS = 30
MIN_IMPRESSIONS = 100
MIN_CLICKS = 5
THRESHOLDS = {
    "ctr": 0.50,                # CTR increased >= 50% vs hist
    "cpc": 0.50,                # CPC increased >= 50%
    "conversion_rate": -0.30,   # CVR decreased >= 30% (negative means drop)
    "cpa": 0.50                 # CPA increased >= 50%
}

# Output
OUTPUT_DIR = "ga_campaign_monitor_output"

# Default email addresses (user requested)
DEFAULT_EMAIL = "ilmazeem3@gmail.com"
EMAIL_FROM = os.getenv("EMAIL_FROM", DEFAULT_EMAIL)
EMAIL_TO = os.getenv("EMAIL_TO", DEFAULT_EMAIL)  # comma-separated allowed

EPS = 1e-9

def send_email(subject, html_body):
    host = os.getenv("EMAIL_SMTP_HOST")
    port = int(os.getenv("EMAIL_SMTP_PORT", "587"))
    user = os.getenv("EMAIL_SMTP_USER")
    pwd = os.getenv("EMAIL_SMTP_PASS")
    from_addr = EMAIL_FROM
    to_addrs = EMAIL_TO

    if not all([host, user, pwd, from_addr, to_addrs]):
        print("SMTP not fully configured via env vars. Email will be skipped. (Set EMAIL_SMTP_* vars)")
        return False

    to_list = [x.strip() for x in to_addrs.split(",") if x.strip()]
    msg = MIMEText(html_body, "html")
    msg["Subject"] = subject
    msg["From"] = from_addr
    msg["To"] = ", ".join(to_list)

    try:
        server = smtplib.SMTP(host, port, timeout=20)
        server.starttls()
        server.login(user, pwd)
        server.sendmail(from_addr, to_list, msg.as_string())
        server.quit()
        print("Email sent to:", to_list)
        return True
    except Exception as e:
        print("Failed to send email:", e)
        return False

# ---------- Google Ads extraction ----------
def get_google_ads_client():
    if GoogleAdsClient is None:
        raise ImportError("google-ads library not installed. Run: pip install google-ads")
    # First try loading from default yaml
    try:
        client = GoogleAdsClient.load_from_storage()
        return client
    except Exception:
        # Optionally try to build client from environment vars (simple)
        cfg = {}
        # required env vars: GOOGLE_ADS_DEVELOPER_TOKEN, GOOGLE_ADS_CLIENT_ID, GOOGLE_ADS_CLIENT_SECRET, GOOGLE_ADS_REFRESH_TOKEN
        token = os.getenv("GOOGLE_ADS_DEVELOPER_TOKEN")
        cid = os.getenv("GOOGLE_ADS_CLIENT_ID")
        secret = os.getenv("GOOGLE_ADS_CLIENT_SECRET")
        refresh = os.getenv("GOOGLE_ADS_REFRESH_TOKEN")
        login_cust = os.getenv("GOOGLE_ADS_LOGIN_CUSTOMER_ID")  # optional
        if not all([token, cid, secret, refresh]):
            raise RuntimeError("Google Ads credentials not found. Use google-ads.yaml or set env vars: GOOGLE_ADS_DEVELOPER_TOKEN, GOOGLE_ADS_CLIENT_ID, GOOGLE_ADS_CLIENT_SECRET, GOOGLE_ADS_REFRESH_TOKEN")
        cfg["developer_token"] = token
        cfg["client_id"] = cid
        cfg["client_secret"] = secret
        cfg["refresh_token"] = refresh
        if login_cust:
            cfg["login_customer_id"] = login_cust
        # build minimal config dict structure expected by client library
        client = GoogleAdsClient.load_from_dict({"developer_token": token,
                                                 "client_id": cid,
                                                 "client_secret": secret,
                                                 "refresh_token": refresh,
                                                 "login_customer_id": login_cust} )
        return client

def fetch_google_ads_data(customer_ids, lookback_days=LOOKBACK_DAYS):
    client = get_google_ads_client()
    ga_service = client.get_service("GoogleAdsService")
    rows = []
    end_date = (datetime.utcnow().date() - timedelta(days=1))  # up to yesterday
    start_date = end_date - timedelta(days=lookback_days - 1)
    # GAQL: campaign resource supports campaign fields + metrics + segments.date
    gaql = f"""
      SELECT
        customer.id,
        campaign.id,
        campaign.name,
        segments.date,
        metrics.impressions,
        metrics.clicks,
        metrics.cost_micros,
        metrics.conversions,
        metrics.conversions_value
      FROM campaign
      WHERE segments.date BETWEEN '{start_date.isoformat()}' AND '{end_date.isoformat()}'
      ORDER BY segments.date
    """
    for cust in customer_ids:
        # google-ads client expects a string customer_id with no dashes
        cust_str = cust.replace("-", "").strip()
        response = ga_service.search(customer_id=cust_str, query=gaql, page_size=10000)
        for result in response:
            # result is a GoogleAdsRow protobuf object; access fields carefully
            cust_id = getattr(result.customer, "id", None)
            campaign = getattr(result, "campaign", None)
            seg_date = getattr(result.segments, "date", None)
            impressions = getattr(result.metrics, "impressions", 0)
            clicks = getattr(result.metrics, "clicks", 0)
            cost_micros = getattr(result.metrics, "cost_micros", 0)
            conversions = getattr(result.metrics, "conversions", 0)
            conv_value = getattr(result.metrics, "conversions_value", 0)
            campaign_id = getattr(campaign, "id", None)
            campaign_name = getattr(campaign, "name", None)
            rows.append({
                "customer_id": int(cust_id) if cust_id is not None else cust_str,
                "date": pd.to_datetime(str(seg_date)),
                "campaign_id": int(campaign_id) if campaign_id is not None else None,
                "campaign_name": campaign_name,
                "impressions": int(impressions) if impressions is not None else 0,
                "clicks": int(clicks) if clicks is not None else 0,
                "cost": float(cost_micros) / 1_000_000.0 if cost_micros is not None else 0.0,
                "conversions": float(conversions) if conversions is not None else 0.0,
                "conversion_value": float(conv_value) if conv_value is not None else 0.0
            })
    if not rows:
        return pd.DataFrame()
    df = pd.DataFrame(rows)
    # normalize date column
    df["date"] = pd.to_datetime(df["date"]).dt.normalize()
    return df

# ---------- Simple monitor logic (same as previous simple version) ----------
def safe_div(a, b):
    return np.where(b == 0, 0.0, a / b)

def compute_daily_metrics(df):
    group = ["date", "customer_id", "campaign_id", "campaign_name"]
    agg = df.groupby(group, as_index=False).agg({
        "impressions": "sum",
        "clicks": "sum",
        "cost": "sum",
        "conversions": "sum",
        "conversion_value": "sum"
    })
    agg["ctr"] = safe_div(agg["clicks"], agg["impressions"])
    agg["cpc"] = safe_div(agg["cost"], agg["clicks"])
    agg["conversion_rate"] = safe_div(agg["conversions"], agg["clicks"])
    agg["cpa"] = safe_div(agg["cost"], agg["conversions"])
    agg["roi"] = safe_div(agg["conversion_value"], agg["cost"])
    return agg

def historical_average(metrics_df, target_date, days):
    hist_end = pd.to_datetime(target_date) - pd.Timedelta(days=1)
    hist_start = hist_end - pd.Timedelta(days=days - 1)
    hist = metrics_df[(metrics_df["date"] >= hist_start) & (metrics_df["date"] <= hist_end)]
    if hist.empty:
        return pd.DataFrame()
    cols = ["customer_id", "campaign_id"]
    hist_avg = hist.groupby(cols).agg({
        "impressions": "mean", "clicks": "mean", "cost": "mean",
        "conversions": "mean", "conversion_value": "mean",
        "ctr": "mean", "cpc": "mean", "conversion_rate": "mean", "cpa": "mean", "roi": "mean"
    }).reset_index()
    hist_avg = hist_avg.rename(columns={c: f"{c}_hist" for c in hist_avg.columns if c not in cols})
    return hist_avg

def detect_breaches(target_df, hist_avg):
    merged = target_df.merge(hist_avg, on=["customer_id", "campaign_id"], how="left")
    merged["ctr_pct"] = (merged["ctr"] - merged.get("ctr_hist", 0)) / (merged.get("ctr_hist", 0) + EPS)
    merged["cpc_pct"] = (merged["cpc"] - merged.get("cpc_hist", 0)) / (merged.get("cpc_hist", 0) + EPS)
    merged["conv_rate_pct"] = (merged["conversion_rate"] - merged.get("conversion_rate_hist", 0)) / (merged.get("conversion_rate_hist", 0) + EPS)
    merged["cpa_pct"] = (merged["cpa"] - merged.get("cpa_hist", 0)) / (merged.get("cpa_hist", 0) + EPS)

    alerts = []
    for _, r in merged.iterrows():
        if r["impressions"] < MIN_IMPRESSIONS or r["clicks"] < MIN_CLICKS:
            continue
        breach_items = []
        if "ctr" in THRESHOLDS and pd.notnull(r.get("ctr_hist")) and r["ctr_pct"] >= THRESHOLDS["ctr"]:
            breach_items.append(("ctr", r["ctr"], r.get("ctr_hist"), r["ctr_pct"]))
        if "cpc" in THRESHOLDS and pd.notnull(r.get("cpc_hist")) and r["cpc_pct"] >= THRESHOLDS["cpc"]:
            breach_items.append(("cpc", r["cpc"], r.get("cpc_hist"), r["cpc_pct"]))
        if "conversion_rate" in THRESHOLDS and pd.notnull(r.get("conversion_rate_hist")):
            th = THRESHOLDS["conversion_rate"]
            if th < 0 and r["conv_rate_pct"] <= th:
                breach_items.append(("conversion_rate_drop", r["conversion_rate"], r.get("conversion_rate_hist"), r["conv_rate_pct"]))
            elif th >= 0 and r["conv_rate_pct"] >= th:
                breach_items.append(("conversion_rate_increase", r["conversion_rate"], r.get("conversion_rate_hist"), r["conv_rate_pct"]))
        if "cpa" in THRESHOLDS and pd.notnull(r.get("cpa_hist")) and r["cpa_pct"] >= THRESHOLDS["cpa"]:
            breach_items.append(("cpa", r["cpa"], r.get("cpa_hist"), r["cpa_pct"]))
        if breach_items:
            alerts.append({
                "customer_id": r["customer_id"],
                "campaign_id": r["campaign_id"],
                "campaign_name": r["campaign_name"],
                "date": pd.to_datetime(r["date"]).date().isoformat(),
                "impressions": int(r["impressions"]),
                "clicks": int(r["clicks"]),
                "cost": float(r["cost"]),
                "conversions": float(r["conversions"]),
                "conversion_value": float(r["conversion_value"]),
                "breach_details": "; ".join([f"{m}: {cur:.4g} vs {hist:.4g} ({pct:+.1%})" for m, cur, hist, pct in breach_items])
            })
    return pd.DataFrame(alerts), merged

def export_and_email(daily_all, daily_target, alerts_df, latest_date):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    ds = latest_date.strftime("%Y%m%d")
    daily_all.to_csv(os.path.join(OUTPUT_DIR, f"daily_all_{ds}.csv"), index=False)
    daily_target.to_csv(os.path.join(OUTPUT_DIR, f"daily_target_{ds}.csv"), index=False)
    alerts_df.to_csv(os.path.join(OUTPUT_DIR, f"alerts_{ds}.csv"), index=False)
    # excel workbook
    try:
        excel_path = os.path.join(OUTPUT_DIR, f"campaign_report_{ds}.xlsx")
        with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
            daily_all.to_excel(writer, sheet_name="daily_all", index=False)
            daily_target.to_excel(writer, sheet_name="daily_target", index=False)
            alerts_df.to_excel(writer, sheet_name="alerts", index=False)
    except Exception as e:
        print("Excel export error (openpyxl?), continuing:", e)

    # Email subject: one day before latest_date (as requested)
    subject_date = (latest_date - timedelta(days=1)).date().isoformat()
    subject = f"Campaign report alerts - {subject_date}"
    if alerts_df.empty:
        body = f"<p>Hi Team,</p><p>No campaign breaches detected for {latest_date.date()}.</p><p>Reports saved to: {os.path.abspath(OUTPUT_DIR)}</p>"
    else:
        body = f"<p>Hi Team,</p><p>Campaign breaches detected for {latest_date.date()}. See table below and exported files.</p>"
        body += alerts_df.to_html(index=False, escape=False)
        body += f"<p>Exports saved to: {os.path.abspath(OUTPUT_DIR)}</p>"

    send_email(subject, body)
    print("Exported files and attempted email send. Output folder:", os.path.abspath(OUTPUT_DIR))

# ---------- Runner ----------
def run():
    if not CUSTOMER_IDS or CUSTOMER_IDS[0].startswith("INSERT"):
        print("Please set CUSTOMER_IDS at top of the script to your Google Ads customer id(s). Exiting.")
        return

    print("Fetching data from Google Ads...")
    df = fetch_google_ads_data(CUSTOMER_IDS, LOOKBACK_DAYS)
    if df.empty:
        print("No data returned from Google Ads. Exiting.")
        return

    print("Computing daily metrics...")
    daily = compute_daily_metrics(df)
    latest_date = daily["date"].max()
    print("Latest date in data:", latest_date.date())

    target_day = daily[daily["date"] == latest_date].copy()
    if target_day.empty:
        print("No rows for latest date; exiting.")
        return

    hist_avg = historical_average(daily, latest_date, HIST_DAYS)
    alerts_df, merged = detect_breaches(target_day, hist_avg)
    export_and_email(daily, target_day, alerts_df, latest_date)
    print("Done.")

if __name__ == "__main__":
    run()
